# 패키지 로딩

In [2]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# 데이터 로딩 및 전처리

## 독립변수

### 구리 가격 (일별 -> 월별)

In [19]:
# 데이터 로딩
copper = pd.read_csv("./구리 공급/(공급) copper-prices-historical-chart-data.csv")

# 날짜데이터 pandas datetime으로 변경
copper["date"] = pd.to_datetime(copper["date"])

# 2000년 1월부터 2023년 3월까지 데이터만 가져오기
copper = copper[copper["date"] >= "2000-01-01"][copper["date"] <= "2023-03-31"].reset_index(drop=True)

# 년 - 월 컬럼 추가 후 월별 평균값 계산
copper['month_year'] = copper['date'].dt.to_period('M')
copper = copper.groupby("month_year").mean("value").reset_index(drop=True)
copper.columns = ["Copper price"]

copper

,Copper price
0,0.855289
1,0.827855
2,0.805130
3,0.776305
4,0.831318
...,...
274,3.656582
275,3.817891
276,4.118691
277,4.082400


### 원유 가격 (일별 -> 월별)

In [23]:
# 데이터 로딩
oil = pd.read_csv("./구리 공급/(공급) Crude_Oil_Spot_Prices_EIA.csv")

# 날짜데이터 pandas datetime으로 변경
oil["Date"] = pd.to_datetime(oil["Date"])

# 2000년 1월부터 2023년 3월까지 데이터만 가져오기
oil = oil[oil["Date"] >= "1995-01-01"][oil["Date"] <= "2023-03-31"].reset_index(drop=True)

# 년 - 월 컬럼 추가 후 월별 평균값 계산
oil['month_year'] = oil['Date'].dt.to_period('M')
oil = oil.groupby("month_year").mean("value").reset_index(drop=True)
oil.columns = ["Oil Price"]

oil

,Oil Price
0,25.511000
1,27.775714
2,27.486087
3,22.764444
4,27.737619
...,...
274,91.415455
275,80.924000
276,82.501429
277,82.585000


### 재고 (월별)

In [25]:
# 데이터 로딩
GRCI = pd.read_csv("./구리 공급/(공급) LME Copper Stock_JAN_2000-MAY_2023.csv")

# 열 이름 변경
GRCI.columns = ["Date", "Stock"]

# 날짜데이터 pandas datetime으로 변경
GRCI["Date"] = pd.to_datetime(GRCI["Date"])

# 쉼표 제거 후 정수형으로 변환
temp_list = []

for i in GRCI["Stock"]:
    temp_list.append(int(i.replace(",","")))
    
GRCI["Stock"] = temp_list

# 필요 없는 행 제거 (2023년 4월, 5월)
GRCI = GRCI[:-2]

GRCI

,Date,Stock
0,2000-01-01,15868200
1,2000-02-01,16834150
2,2000-03-01,18638525
3,2000-04-01,13103150
4,2000-05-01,13712200
...,...,...
274,2022-11-01,1971200
275,2022-12-01,1696075
276,2023-01-01,1719650
277,2023-02-01,1318875


### 생산비용 (분기별 -> 월별)

In [14]:
# 데이터 로딩
cost = pd.read_csv("./구리 공급/(공급) Antofagasta CASH COSTS & Transport.csv")

# 열 이름 변경
cost.columns = ["Date", "Cash Costs", "Transport"]

# 날짜데이터 pandas datetime으로 변경
cost["Date"] = pd.to_datetime(cost["Date"])

# 분기별 데이터 월별 데이터로 늘리기
## 빈 DataFrame 생성
temp_df = pd.DataFrame()

## DataFrame 행 길이만큼 반복하여 각 행의 월에 따라 실행되는 조건문을 설정
for i in range(len(cost)):
    if cost.iloc[i]["Date"].month == 1:
        year = cost.iloc[i]["Date"].year
        
        # 행 3개를 행 단위로 추가하고, Date를 해당 월, 다음달, 다다음달로 변경
        temp = pd.DataFrame([cost.iloc[i], cost.iloc[i], cost.iloc[i]])
        temp["Date"] = [f'{year}-01-01', f'{year}-02-01', f'{year}-03-01']
        
        # 생성한 DataFrame을 임시 DataFrame에 추가
        temp_df = temp_df.append(temp)
        
    if cost.iloc[i]["Date"].month == 4:
        year = cost.iloc[i]["Date"].year
        temp = pd.DataFrame([cost.iloc[i], cost.iloc[i], cost.iloc[i]])
        temp["Date"] = [f'{year}-04-01', f'{year}-05-01', f'{year}-06-01']
        temp_df = temp_df.append(temp)
        
    if cost.iloc[i]["Date"].month == 7:
        year = cost.iloc[i]["Date"].year
        temp = pd.DataFrame([cost.iloc[i], cost.iloc[i], cost.iloc[i]])
        temp["Date"] = [f'{year}-07-01', f'{year}-08-01', f'{year}-09-01']
        temp_df = temp_df.append(temp)
        
    if cost.iloc[i]["Date"].month == 10:
        year = cost.iloc[i]["Date"].year
        temp = pd.DataFrame([cost.iloc[i], cost.iloc[i], cost.iloc[i]])
        temp["Date"] = [f'{year}-10-01', f'{year}-11-01', f'{year}-12-01']
        temp_df = temp_df.append(temp)

# 생성한 월별 데이터의 DataFrame의 행 번호를 초기화        
cost = temp_df.reset_index(drop=True)

# 필요한 데이터만 추출
cost = cost[["Cash Costs", "Transport"]]
cost

,Cash Costs,Transport
0,0.39,928.60
1,0.39,928.60
2,0.39,928.60
3,0.39,966.68
4,0.39,966.68
...,...,...
274,1.27,1811.00
275,1.27,1811.00
276,1.54,1751.00
277,1.54,1751.00


### Chile Copper Mining GDP

In [16]:
# 데이터 로딩
M_GDP = pd.read_csv("./구리 공급/(공급) GDP in Copper Mining Chile.csv")

# 열 이름 변경
M_GDP.columns = ["Date", "M_GDP"]

# 날짜데이터 pandas datetime으로 변경
M_GDP["Date"] = pd.to_datetime(M_GDP["Date"])

# 분기별 데이터 월별 데이터로 늘리기
## 빈 DataFrame 생성
temp_df = pd.DataFrame()

## DataFrame 행 길이만큼 반복하여 각 행의 월에 따라 실행되는 조건문을 설정
for i in range(len(M_GDP)):
    if M_GDP.iloc[i]["Date"].month == 3:
        year = M_GDP.iloc[i]["Date"].year
        
        # 행 3개를 행 단위로 추가하고, Date를 해당 월, 다음달, 다다음달로 변경
        temp = pd.DataFrame([M_GDP.iloc[i], M_GDP.iloc[i], M_GDP.iloc[i]])
        temp["Date"] = [f'{year}-01-01', f'{year}-02-01', f'{year}-03-01']
        
        # 생성한 DataFrame을 임시 DataFrame에 추가
        temp_df = temp_df.append(temp)
        
    if M_GDP.iloc[i]["Date"].month == 6:
        year = M_GDP.iloc[i]["Date"].year
        temp = pd.DataFrame([M_GDP.iloc[i], M_GDP.iloc[i], M_GDP.iloc[i]])
        temp["Date"] = [f'{year}-04-01', f'{year}-05-01', f'{year}-06-01']
        temp_df = temp_df.append(temp)
        
    if M_GDP.iloc[i]["Date"].month == 9:
        year = M_GDP.iloc[i]["Date"].year
        temp = pd.DataFrame([M_GDP.iloc[i], M_GDP.iloc[i], M_GDP.iloc[i]])
        temp["Date"] = [f'{year}-07-01', f'{year}-08-01', f'{year}-09-01']
        temp_df = temp_df.append(temp)
        
    if M_GDP.iloc[i]["Date"].month == 12:
        year = M_GDP.iloc[i]["Date"].year
        temp = pd.DataFrame([M_GDP.iloc[i], M_GDP.iloc[i], M_GDP.iloc[i]])
        temp["Date"] = [f'{year}-10-01', f'{year}-11-01', f'{year}-12-01']
        temp_df = temp_df.append(temp)

# 생성한 월별 데이터의 DataFrame의 행 번호를 초기화    
M_GDP = temp_df.reset_index(drop=True)

# 쉼표 제거 후 정수형으로 변환
temp_list = []

for i in M_GDP["M_GDP"]:
    temp_list.append(float(i.replace(",","")))
    
M_GDP["M_GDP"] = temp_list


M_GDP = M_GDP[["M_GDP"]]
M_GDP

,M_GDP
0,435.48
1,435.48
2,435.48
3,476.12
4,476.12
...,...
274,8414.73
275,8414.73
276,6543.48
277,6543.48


## 종속변수: 전세계 구리 공급

In [17]:
DEP = pd.read_csv("./구리 공급/(종속 변수) 230622 Global Copper Production.csv")
DEP = DEP[60:-2].reset_index(drop=True)[["PX_LAST"]]
DEP

,PX_LAST
0,1202241.000
1,1155760.000
2,1253977.000
3,1186378.000
4,1238011.000
...,...
274,2193243.262
275,2197474.345
276,2306260.603
277,2208886.704


# 데이터 병합

In [31]:
# 구리가격, 생산비용, 재고, 원유가격, Mining GDP, 종속변수 병합
total = pd.concat([copper, cost, GRCI, oil, M_GDP, DEP], axis = 1)

# 열 순서 변경
total = total[['Date', 'Copper price', 'Cash Costs', 'Transport', 'Stock', 'Oil Price', 'M_GDP', 'PX_LAST']]

total

,Date,Copper price,Cash Costs,Transport,Stock,Oil Price,M_GDP,PX_LAST
0,2000-01-01,0.855289,0.39,928.60,15868200,25.511000,435.48,1202241.000
1,2000-02-01,0.827855,0.39,928.60,16834150,27.775714,435.48,1155760.000
2,2000-03-01,0.805130,0.39,928.60,18638525,27.486087,435.48,1253977.000
3,2000-04-01,0.776305,0.39,966.68,13103150,22.764444,476.12,1186378.000
4,2000-05-01,0.831318,0.39,966.68,13712200,27.737619,476.12,1238011.000
...,...,...,...,...,...,...,...,...
274,2022-11-01,3.656582,1.27,1811.00,1971200,91.415455,8414.73,2193243.262
275,2022-12-01,3.817891,1.27,1811.00,1696075,80.924000,8414.73,2197474.345
276,2023-01-01,4.118691,1.54,1751.00,1719650,82.501429,6543.48,2306260.603
277,2023-02-01,4.082400,1.54,1751.00,1318875,82.585000,6543.48,2208886.704


# 데이터 저장

In [30]:
total.to_csv("./구리 공급/공급 변수_최종.csv", index=False)